In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as vdatasets
import torchvision.utils as vutils
from tensorboardX import SummaryWriter

torch.manual_seed(1)

# 1. MNIST데이터

In [3]:
BATCH_SIZE= 64
train_dataset = vdatasets.MNIST(root='../data/MNIST/',
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=False)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True,
                                           num_workers=2)

test_dataset = vdatasets.MNIST(root='../data/MNIST/',
                               train=False, 
                               transform=transforms.ToTensor(),
                               download=True)


test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=BATCH_SIZE, 
                                           shuffle=False,
                                           num_workers=2)

# 2. 모델

Training vs Evaluation  

Before training the model, it is imperative to call model.train(). Likewise, you must call model.eval() before testing the model. This corrects for the differences in dropout, batch normalization during training and testing.

In [4]:
class NN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_p=0.5):
        super(NN, self).__init__()
        self.linear1= nn.Linear(input_size,hidden_size)
        self.linear2= nn.Linear(hidden_size, hidden_size)
        self.linear3= nn.Linear(hidden_size, output_size)
        
        self.dropout= nn.Dropout(dropout_p)
        
    def forward(self, inputs):
        outputs= F.relu(self.linear1(inputs))
        outputs= self.dropout(outputs)
        outputs= F.relu(self.linear2(outputs))
        outputs= self.dropout(outputs)
        return self.linear3(outputs)

In [5]:
INPUT_SIZE=train_dataset[0][0].size()[1]**2
HIDDEN_SIZE= 512
OUTPUT_SIZE=10

model= NN(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)

model.train() # train process

model.eval() # test process


NN(
  (linear1): Linear(in_features=784, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=512, bias=True)
  (linear3): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.5)
)

# 3. 학습

In [6]:
EPOCH=15
LR=0.1
INPUT_SIZE=train_dataset[0][0].size()[1]**2
HIDDEN_SIZE= 512
OUTPUT_SIZE=10
BATCH_SIZE=64
NUM_LAYERS=1
DROPOUT=0.5

model= NN(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)

loss_function= nn.CrossEntropyLoss()
optimizer= optim.SGD(model.parameters(), lr=LR)

In [7]:
writer= SummaryWriter('runs/drop_out')

In [11]:
%%time
model.train()
j=0
for epoch in range(EPOCH):
    losses=[]
    for i, (inputs, targets) in enumerate(train_loader):
        model.zero_grad()
        pred= model(inputs.view(len(inputs),-1))
        loss= loss_function(pred, targets)
        loss.backward()
        optimizer.step()
        losses.append(loss.data.tolist())
        writer.add_scalar('data/drop_out', loss.data.tolist(), j)

        j+=1
        if i % 500 == 0:
            print("[%d/%d] [%03d/%d] mean_loss : %.3f" % (epoch,EPOCH,i,len(train_loader),np.mean(losses)))
            losses=[]


[0/15] [000/938] mean_loss : 0.476
[0/15] [500/938] mean_loss : 0.364
[1/15] [000/938] mean_loss : 0.284
[1/15] [500/938] mean_loss : 0.214
[2/15] [000/938] mean_loss : 0.222
[2/15] [500/938] mean_loss : 0.165
[3/15] [000/938] mean_loss : 0.248
[3/15] [500/938] mean_loss : 0.134
[4/15] [000/938] mean_loss : 0.117
[4/15] [500/938] mean_loss : 0.118
[5/15] [000/938] mean_loss : 0.116
[5/15] [500/938] mean_loss : 0.106
[6/15] [000/938] mean_loss : 0.175
[6/15] [500/938] mean_loss : 0.097
[7/15] [000/938] mean_loss : 0.016
[7/15] [500/938] mean_loss : 0.081
[8/15] [000/938] mean_loss : 0.161
[8/15] [500/938] mean_loss : 0.078
[9/15] [000/938] mean_loss : 0.055
[9/15] [500/938] mean_loss : 0.076
[10/15] [000/938] mean_loss : 0.030
[10/15] [500/938] mean_loss : 0.064
[11/15] [000/938] mean_loss : 0.019
[11/15] [500/938] mean_loss : 0.065
[12/15] [000/938] mean_loss : 0.112
[12/15] [500/938] mean_loss : 0.062
[13/15] [000/938] mean_loss : 0.061
[13/15] [500/938] mean_loss : 0.057
[14/15] [000

# 4. evaluation

In [34]:
def evaluation(data_loader, model):
    model.eval() # for dropout at test time!
    loss_function= nn.CrossEntropyLoss(size_average=False)
    num_equal=0
    losses=0
    for inputs, targets in data_loader:
        pred= model(inputs.view(len(inputs),-1))
        losses += loss_function(pred, targets).data.tolist()
        outputs= pred.max(1)[1] # argmax
        num_equal += torch.eq(outputs, targets).sum().tolist()
        
    return num_equal/ len(data_loader.dataset), losses/len(data_loader.dataset) 

In [45]:
train_acc, train_loss= evaluation(train_loader, model)
test_acc, test_loss= evaluation(test_loader, model)

print('<train acc : > {} / <train loss> : {}'.format(round(train_acc,3), round(train_loss,3)))
print('<test  acc : > {} / <test  loss> : {}'.format(round(test_acc,3), round(test_loss,3)))

<train acc : > 0.996 / <train loss> : 0.015
<test  acc : > 0.984 / <test  loss> : 0.054
